In [1]:
import numpy as np
import pandas as pd
import os
import re
import csv
import torch

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001F930D31A10>>
Traceback (most recent call last):
  File "e:\ComputerScience\AI ML DL\LogParser\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 790, in _clean_thread_parent_frames
    active_threads = {thread.ident for thread in threading.enumerate()}
                                                 ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\krish\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1494, in enumerate
    def enumerate():
    
KeyboardInterrupt: 


In [ ]:
raw_directory = "E:\\ComputerScience\\AI ML DL\\LogParser\\raw_data"
structured_directory = "E:\\ComputerScience\\AI ML DL\\LogParser\\structured_data"

def read_raw(directory): 
  all_logs = []
  for filename in os.listdir(directory):
    if filename.endswith('.txt') or filename.endswith('.log'):
      file_path = os.path.join(directory, filename)
      with open(file_path, 'r') as f:
        logs = f.readlines()
      all_logs.extend(logs)
  number_of_logs = len(all_logs)
  return all_logs

def read_structured(directory):
  all_logs = []
  for filename in os.listdir(directory):
    if filename.endswith('.csv'):
      file_path = os.path.join(directory, filename)
      with open(file_path, 'r') as f:
        csv_reader = csv.reader(f)
        for row in csv_reader:
          log = row[1:]
          all_logs.append(log)
  number_of_logs = len(all_logs)
  return all_logs

raw_logs = read_raw(raw_directory)
structured_logs = read_structured(structured_directory)

print("No of raw logs:",len(raw_logs))
print("No of structured logs:",len(structured_logs))

No of raw logs: 31999
No of structured logs: 32016


In [ ]:
raw_df = pd.DataFrame(raw_logs)
structured_df = pd.DataFrame(structured_logs)

structured_df.columns = structured_df.iloc[0]
structured_df = structured_df.drop(structured_df.index[0])
structured_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32015 entries, 1 to 32015
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Date           32015 non-null  object
 1   Time           32015 non-null  object
 2   Pid            32015 non-null  object
 3   Tid            32015 non-null  object
 4   Level          32015 non-null  object
 5   Component      28013 non-null  object
 6   Content        26012 non-null  object
 7   EventId        22010 non-null  object
 8   EventTemplate  16007 non-null  object
 9   None           8004 non-null   object
 10  None           4002 non-null   object
 11  None           4002 non-null   object
 12  None           2001 non-null   object
dtypes: object(13)
memory usage: 3.2+ MB


In [ ]:
import re

def preprocess_logs(raw_logs, labels):
  """
  Preprocesses a list of raw log messages for multi-task learning.

  Args:
      raw_logs: A list of strings, where each string represents a raw log message.
      labels: A list of strings representing the labels you want to classify.

  Returns:
      A list of lists, where each inner list represents a preprocessed log message
          and a one-hot encoded representation of the corresponding labels.
  """
  processed_logs = []
  timestamp_regex = r"\[(.*?)\]|\d{1,2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}.*|\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}"

  for log in raw_logs:
    # Lowercase and remove punctuation/special characters
    processed_log = re.sub(r"[^\w\s]", "", log.lower())

    # Extract timestamp (optional)
    timestamp_match = re.search(timestamp_regex, log)
    timestamp = timestamp_match.group(1) if timestamp_match else None

    # Extract features (optional)
    # You can add logic here to extract other features based on your data and needs
    # This example extracts component names using a simple heuristic (assuming component names start with uppercase)
    component = None
    component_words = processed_log.split()
    for word in component_words:
      if word[0].isupper():
        component = word
        break  # Limit component to the first uppercase word

    # One-hot encode labels (assuming labels are provided as a separate list)
    one_hot_labels = [0] * len(labels)  # Initialize with zeros
    for label in labels:
      if label in log:  # Check if label is present in the log message
        one_hot_labels[labels.index(label)] = 1  # Set corresponding label to 1

    # Combine preprocessed log, features (if extracted), and one-hot encoded labels
    processed_log_with_features = [processed_log]
    if timestamp:
      processed_log_with_features.append(timestamp)
    if component:
      processed_log_with_features.append(component)
    processed_logs.append(processed_log_with_features + one_hot_labels)

  return processed_logs

# Example usage:
raw_logs = ["- 1131566461 2005.11.09 dn228 Nov 9 12:01:01 dn228/dn228 crond(pam_unix)[2915]: session closed for user roo", "2016-09-28 04:30:31, Info                  CSI    00000002@2016/9/27:20:30:31.458 WcpInitialize (wcp.dll version 0.0.0.6) called (stack @0x7fed806eb5d @0x7fefa006ade @0x7fef9fd2984 @0x7fef9f83665 @0xff83e97c @0xff83d799"]
labels = ["error", "warning", "info", "MyApp", "ComponentX"]  # Example labels including components
processed_logs = preprocess_logs(raw_logs, labels)

print(processed_logs)


[[' 1131566461 20051109 dn228 nov 9 120101 dn228dn228 crondpam_unix2915 session closed for user roo', '2915', 0, 0, 0, 0, 0], ['20160928 043031 info                  csi    000000022016927203031458 wcpinitialize wcpdll version 0006 called stack 0x7fed806eb5d 0x7fefa006ade 0x7fef9fd2984 0x7fef9f83665 0xff83e97c 0xff83d799', 0, 0, 0, 0, 0]]


In [ ]:
windows_df = pd.read_csv("structured_data\Windows_2k.log_structured.csv")
mac_df = pd.read_csv("structured_data/Mac_2k.log_structured.csv")

mac_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   LineId         2000 non-null   int64 
 1   Month          2000 non-null   object
 2   Date           2000 non-null   int64 
 3   Time           2000 non-null   object
 4   User           2000 non-null   object
 5   Component      2000 non-null   object
 6   PID            2000 non-null   int64 
 7   Address        54 non-null     object
 8   Content        2000 non-null   object
 9   EventId        2000 non-null   object
 10  EventTemplate  2000 non-null   object
dtypes: int64(3), object(8)
memory usage: 172.0+ KB


In [ ]:
windows_df.head()

,LineId,Date,Time,Level,Component,Content,EventId,EventTemplate
0,1,2016-09-28,04:30:30,Info,CBS,Loaded Servicing Stack v6.1.7601.23505 with Co...,E23,Loaded Servicing Stack <*> with Core: <*>\cbsc...
1,2,2016-09-28,04:30:31,Info,CSI,00000001@2016/9/27:20:30:31.455 WcpInitialize ...,E13,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...
2,3,2016-09-28,04:30:31,Info,CSI,00000002@2016/9/27:20:30:31.458 WcpInitialize ...,E13,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...
3,4,2016-09-28,04:30:31,Info,CSI,00000003@2016/9/27:20:30:31.458 WcpInitialize ...,E13,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...
4,5,2016-09-28,04:30:31,Info,CBS,Ending TrustedInstaller initialization.,E17,Ending TrustedInstaller initialization.


In [ ]:
mac_df.head()

,LineId,Month,Date,Time,User,Component,PID,Address,Content,EventId,EventTemplate
0,1,Jul,1,09:00:55,calvisitor-10-105-160-95,kernel,0,NaN,IOThunderboltSwitch<0>(0x0)::listenerCallback ...,E252,IOThunderboltSwitch<<*>>(<*>)::listenerCallbac...
1,2,Jul,1,09:01:05,calvisitor-10-105-160-95,com.apple.CDScheduler,43,NaN,Thermal pressure state: 1 Memory pressure stat...,E323,Thermal pressure state: <*> Memory pressure st...
2,3,Jul,1,09:01:06,calvisitor-10-105-160-95,QQ,10018,NaN,FA||Url||taskID[2019352994] dealloc,E216,FA||Url||taskID[<*>] dealloc
3,4,Jul,1,09:02:26,calvisitor-10-105-160-95,kernel,0,NaN,ARPT: 620701.011328: AirPort_Brcm43xx::syncPow...,E128,ARPT: <*>.<*>: AirPort_<*>::syncPowerState: WW...
4,5,Jul,1,09:02:26,authorMacBook-Pro,kernel,0,NaN,ARPT: 620702.879952: AirPort_Brcm43xx::platfor...,E124,ARPT: <*>.<*>: AirPort_<*>::platformWoWEnable:...


In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
raw_directory = "E:\\ComputerScience\\AI ML DL\\LogParser\\raw_data"
structured_directory = "E:\\ComputerScience\\AI ML DL\\LogParser\\structured_data"

def read_raw(directory): 
  all_logs = []
  for filename in os.listdir(directory):
    if filename.endswith('.txt') or filename.endswith('.log'):
      file_path = os.path.join(directory, filename)
      with open(file_path, 'r') as f:
        logs = f.readlines()
      all_logs.extend(logs)
  number_of_logs = len(all_logs)
  return all_logs

def read_structured(directory):
  all_logs = []
  for filename in os.listdir(directory):
    if filename.endswith('.csv'):
      file_path = os.path.join(directory, filename)
      df = pd.read_csv(file_path)
      all_logs.append(df)
  return all_logs

raw_logs = read_raw(raw_directory)
structured_logs = read_structured(structured_directory)

print("No of raw logs:",len(raw_logs))

No of raw logs: 31999


In [ ]:
def preprocess_logs_for_ner(raw_logs):
    preprocessed_logs = []
    for log in raw_logs:
        cleaned_log = re.sub(r'[^\w\s]', '', log.lower())
        preprocessed_logs.append(cleaned_log)
    return preprocessed_logs

for df in structured_logs:
    df['preprocessed_content'] = preprocess_logs_for_ner(df['Content'].tolist())

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class LogsDataset(Dataset):
    def __init__(self, texts, entities, tokenizer, labels_map):
        self.texts = texts
        self.entities = entities
        self.tokenizer = tokenizer
        self.labels_map = labels_map

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        entities = self.entities[idx]

        # Tokenize text
        tokenized_inputs = self.tokenizer(text, padding='max_length', truncation=True, return_tensors="pt")
        input_ids = tokenized_inputs['input_ids'].squeeze()
        attention_mask = tokenized_inputs['attention_mask'].squeeze()
        
        # Align labels with tokens
        labels = [-100] * len(input_ids)  # Initialize with -100 for ignored tokens
        for start, end, entity_label in entities:
            for i in range(start, end):
                if i < len(labels):
                    labels[i] = self.labels_map[entity_label]
        
        labels = torch.tensor(labels, dtype=torch.long)

        return input_ids, attention_mask, labels

In [ ]:
def convert_to_dataset(df, entity_columns, tokenizer, labels_map):
    """
    Convert DataFrame to a PyTorch Dataset.

    Args:
        df: A Pandas DataFrame containing log data and entity labels.
        entity_columns: A list of strings representing the columns with entity labels.
        tokenizer: The tokenizer instance.
        labels_map: A dictionary mapping label names to label indices.

    Returns:
        A PyTorch Dataset.
    """
    texts = df['preprocessed_content'].tolist()
    entities = []
    for _, row in df.iterrows():
        entity_list = []
        for col in entity_columns:
            if pd.notna(row[col]):
                start, end, label = row[col].split('|')
                entity_list.append((int(start), int(end), label))
        entities.append(entity_list)
    
    return LogsDataset(texts, entities, tokenizer, labels_map)

# Define entity columns (adjust based on your CSV structure)
entity_columns = ['LineId','Date','Time','Level','Component','Content','EventId','EventTemplate']

labels = ["O", "B-TIMESTAMP", "I-TIMESTAMP", "B-ERROR_TYPE", "I-ERROR_TYPE", "B-USER_ID", "I-USER_ID", "B-IP_ADDRESS", "I-IP_ADDRESS"]
labels_map = {label: i for i, label in enumerate(labels)}

# Convert each dataset
pytorch_datasets = [convert_to_dataset(df, entity_columns, tokenizer, labels_map) for df in structured_logs]

In [ ]:
structured_df.head()

,Date,Time,Pid,Tid,Level,Component,Content,EventId,EventTemplate,None,None,None,None
1,03-17,16:13:38.811,1702,2395,D,WindowManager,printFreezingDisplayLogsopening app wtoken = A...,E100,printFreezingDisplayLogsopening app wtoken = A...,None,None,None,None
2,03-17,16:13:38.819,1702,8671,D,PowerManagerService,"acquire lock=233570404, flags=0x1, tag=""View L...",E10,"acquire lock=<*>, flags=<*>, tag=""<*>"", name=<...",None,None,None,None
3,03-17,16:13:38.820,1702,8671,D,PowerManagerService,"ready=true,policy=3,wakefulness=1,wksummary=0x...",E103,"ready=true,policy=<*>,wakefulness=<*>,wksummar...",None,None,None,None
4,03-17,16:13:38.839,1702,2113,V,WindowManager,Skipping AppWindowToken{df0798e token=Token{78...,E131,Skipping AppWindowToken{<*> token=Token{<*> Ac...,None,None,None,None
5,03-17,16:13:38.859,2227,2227,D,TextView,visible is system.time.showampm,E165,visible is <*>,None,None,None,None


In [ ]:
structured_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32015 entries, 1 to 32015
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Date           32015 non-null  object
 1   Time           32015 non-null  object
 2   Pid            32015 non-null  object
 3   Tid            32015 non-null  object
 4   Level          32015 non-null  object
 5   Component      28013 non-null  object
 6   Content        26012 non-null  object
 7   EventId        22010 non-null  object
 8   EventTemplate  16007 non-null  object
 9   None           8004 non-null   object
 10  None           4002 non-null   object
 11  None           4002 non-null   object
 12  None           2001 non-null   object
dtypes: object(13)
memory usage: 3.2+ MB


In [ ]:
structured_df = structured_df.iloc[:, :-8]
structured_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32015 entries, 1 to 32015
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    32015 non-null  object
 1   Time    32015 non-null  object
 2   Pid     32015 non-null  object
 3   Tid     32015 non-null  object
 4   Level   32015 non-null  object
dtypes: object(5)
memory usage: 1.2+ MB


In [ ]:
import re
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForTokenClassification, TrainingArguments, Trainer

In [ ]:
raw_directory = "E:\\ComputerScience\\AI ML DL\\LogParser\\raw_data"

def read_raw(directory): 
  all_logs = []
  for filename in os.listdir(directory):
    if filename.endswith('.txt') or filename.endswith('.log'):
      file_path = os.path.join(directory, filename)
      with open(file_path, 'r') as f:
        logs = f.readlines()
      all_logs.extend(logs)
  number_of_logs = len(all_logs)
  return all_logs

datasets = read_raw(raw_directory)
print("No of raw logs:",len(raw_logs))

No of raw logs: 31999


In [ ]:
def preprocess_logs_for_ner(raw_logs):
    preprocessed_logs = []
    for log in raw_logs:
        cleaned_log = re.sub(r'[^\w\s]', '', log.lower())
        preprocessed_logs.append(cleaned_log)
    return preprocessed_logs

In [ ]:
class LogsDataset(Dataset):
    def __init__(self, texts, entities, tokenizer, labels_map):
        self.texts = texts
        self.entities = entities
        self.tokenizer = tokenizer
        self.labels_map = labels_map

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        entities = self.entities[idx]

        tokenized_inputs = self.tokenizer(text, padding='max_length', truncation=True, return_tensors="pt")
        input_ids = tokenized_inputs['input_ids'].squeeze()
        attention_mask = tokenized_inputs['attention_mask'].squeeze()
        
        labels = [-100] * len(input_ids)
        for start, end, entity_label in entities:
            for i in range(start, end):
                if i < len(labels):
                    labels[i] = self.labels_map[entity_label]
        
        labels = torch.tensor(labels, dtype=torch.long)

        return input_ids, attention_mask, labels

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer,BertForTokenClassification, BertForSequenceClassification, BertTokenizerFast, AdamW
from torch.utils.data import DataLoader, TensorDataset, Dataset
import re

In [ ]:
struct_df = pd.read_csv("structured_data\Windows_2k.log_structured.csv")
struct_df.head()

,LineId,Date,Time,Level,Component,Content,EventId,EventTemplate
0,1,2016-09-28,04:30:30,Info,CBS,Loaded Servicing Stack v6.1.7601.23505 with Co...,E23,Loaded Servicing Stack <*> with Core: <*>\cbsc...
1,2,2016-09-28,04:30:31,Info,CSI,00000001@2016/9/27:20:30:31.455 WcpInitialize ...,E13,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...
2,3,2016-09-28,04:30:31,Info,CSI,00000002@2016/9/27:20:30:31.458 WcpInitialize ...,E13,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...
3,4,2016-09-28,04:30:31,Info,CSI,00000003@2016/9/27:20:30:31.458 WcpInitialize ...,E13,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...
4,5,2016-09-28,04:30:31,Info,CBS,Ending TrustedInstaller initialization.,E17,Ending TrustedInstaller initialization.


In [ ]:
import pandas as pd

def custom_parser(data):
    lines = data.split('\n')  # Split by newlines
    cleaned_lines = [line.strip() for line in lines if line.strip()]  # Remove leading/trailing whitespace and empty lines
    return cleaned_lines

# Read the file content as a single string
with open("raw_data/Windows_2k.log", 'r') as file:
    data = file.read()

# Apply the custom parser
cleaned_lines = custom_parser(data)

# Create a DataFrame from the cleaned lines
df = pd.DataFrame(cleaned_lines, columns=['log'])

df.head()

,log
0,"2016-09-28 04:30:30, Info CBS..."
1,"2016-09-28 04:30:31, Info CSI..."
2,"2016-09-28 04:30:31, Info CSI..."
3,"2016-09-28 04:30:31, Info CSI..."
4,"2016-09-28 04:30:31, Info CBS..."


In [ ]:
struct_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   LineId         2000 non-null   int64 
 1   Date           2000 non-null   object
 2   Time           2000 non-null   object
 3   Level          2000 non-null   object
 4   Component      2000 non-null   object
 5   Content        2000 non-null   object
 6   EventId        2000 non-null   object
 7   EventTemplate  2000 non-null   object
dtypes: int64(1), object(7)
memory usage: 125.1+ KB


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   log     2000 non-null   object
dtypes: object(1)
memory usage: 15.8+ KB


In [ ]:
combined_df = pd.concat([struct_df, df], axis=1)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
combined_df['log']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
1995    NaN
1996    NaN
1997    NaN
1998    NaN
1999    NaN
Name: log, Length: 12000, dtype: object

In [ ]:
struct_df['log'] = np.nan
combined_df = pd.concat([struct_df, df], axis=0)
combined_df.head()

,LineId,Date,Time,Level,Component,Content,EventId,EventTemplate,log
0,1.0,2016-09-28,04:30:30,Info,CBS,Loaded Servicing Stack v6.1.7601.23505 with Co...,E23,Loaded Servicing Stack <*> with Core: <*>\cbsc...,NaN
1,2.0,2016-09-28,04:30:31,Info,CSI,00000001@2016/9/27:20:30:31.455 WcpInitialize ...,E13,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...,NaN
2,3.0,2016-09-28,04:30:31,Info,CSI,00000002@2016/9/27:20:30:31.458 WcpInitialize ...,E13,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...,NaN
3,4.0,2016-09-28,04:30:31,Info,CSI,00000003@2016/9/27:20:30:31.458 WcpInitialize ...,E13,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...,NaN
4,5.0,2016-09-28,04:30:31,Info,CBS,Ending TrustedInstaller initialization.,E17,Ending TrustedInstaller initialization.,NaN


In [ ]:
struct_df_reset = struct_df.reset_index(drop=True)  # Drop the old index
df_reset = df.reset_index(drop=True) 

combined_df = pd.concat([df_reset,struct_df_reset], axis=1)
combined_df.head()

,log,LineId,Date,Time,Level,Component,Content,EventId,EventTemplate
0,"2016-09-28 04:30:30, Info CBS...",1,2016-09-28,04:30:30,Info,CBS,Loaded Servicing Stack v6.1.7601.23505 with Co...,E23,Loaded Servicing Stack <*> with Core: <*>\cbsc...
1,"2016-09-28 04:30:31, Info CSI...",2,2016-09-28,04:30:31,Info,CSI,00000001@2016/9/27:20:30:31.455 WcpInitialize ...,E13,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...
2,"2016-09-28 04:30:31, Info CSI...",3,2016-09-28,04:30:31,Info,CSI,00000002@2016/9/27:20:30:31.458 WcpInitialize ...,E13,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...
3,"2016-09-28 04:30:31, Info CSI...",4,2016-09-28,04:30:31,Info,CSI,00000003@2016/9/27:20:30:31.458 WcpInitialize ...,E13,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...
4,"2016-09-28 04:30:31, Info CBS...",5,2016-09-28,04:30:31,Info,CBS,Ending TrustedInstaller initialization.,E17,Ending TrustedInstaller initialization.


In [ ]:
struct_df.head()

,LineId,Date,Time,Level,Component,Content,EventId,EventTemplate
0,1,2016-09-28,04:30:30,Info,CBS,Loaded Servicing Stack v6.1.7601.23505 with Co...,E23,Loaded Servicing Stack <*> with Core: <*>\cbsc...
1,2,2016-09-28,04:30:31,Info,CSI,00000001@2016/9/27:20:30:31.455 WcpInitialize ...,E13,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...
2,3,2016-09-28,04:30:31,Info,CSI,00000002@2016/9/27:20:30:31.458 WcpInitialize ...,E13,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...
3,4,2016-09-28,04:30:31,Info,CSI,00000003@2016/9/27:20:30:31.458 WcpInitialize ...,E13,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...
4,5,2016-09-28,04:30:31,Info,CBS,Ending TrustedInstaller initialization.,E17,Ending TrustedInstaller initialization.


In [ ]:
df.head()

,log
0,"2016-09-28 04:30:30, Info CBS..."
1,"2016-09-28 04:30:31, Info CSI..."
2,"2016-09-28 04:30:31, Info CSI..."
3,"2016-09-28 04:30:31, Info CSI..."
4,"2016-09-28 04:30:31, Info CBS..."


In [ ]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   log            2000 non-null   object
 1   LineId         2000 non-null   int64 
 2   Date           2000 non-null   object
 3   Time           2000 non-null   object
 4   Level          2000 non-null   object
 5   Component      2000 non-null   object
 6   Content        2000 non-null   object
 7   EventId        2000 non-null   object
 8   EventTemplate  2000 non-null   object
dtypes: int64(1), object(8)
memory usage: 140.8+ KB


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

In [ ]:
def preprocess_df(df):
    # Convert Date and Time to timestamp
    df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
    df['Timestamp'] = df['DateTime'].astype('int64') // 10**9
    
    # Encode categorical variables
    le = LabelEncoder()
    categorical_columns = ['Level', 'Component', 'EventId', 'EventTemplate']
    for col in categorical_columns:
        df[col + '_encoded'] = le.fit_transform(df[col])
    
    # Select features for labels
    label_columns = ['LineId', 'Timestamp', 'Level_encoded', 'Component_encoded', 'EventId_encoded', 'EventTemplate_encoded']
    
    return df, label_columns

In [ ]:
df, label_columns = preprocess_df(df)

In [ ]:
class LogDataset(Dataset):
    def __init__(self, logs, labels, tokenizer, max_len):
        self.logs = logs
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.logs)
    
    def __getitem__(self, item):
        log = str(self.logs[item])
        labels = self.labels[item]
        
        encoding = self.tokenizer.encode_plus(
            log,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        return {
            'log_text': log,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.FloatTensor(labels)
        }

In [ ]:
logs = df['log'].tolist()
labels = df[label_columns].values.tolist()

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(labels[0]))

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
dataset = LogDataset(logs, labels, tokenizer, max_len=128)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5)

e:\ComputerScience\AI ML DL\LogParser\.venv\Lib\site-packages\transformers\optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    for batch in dataloader:
        optimizer.zero_grad()
        inputs = {'input_ids': batch['input_ids'], 'attention_mask': batch['attention_mask'], 'labels': batch['labels']}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

In [ ]:
model.save_pretrained('./log_analysis_model')

In [ ]:
def predict_log_details(log_text):
    inputs = tokenizer(log_text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    predictions = outputs.logits.sigmoid()
    return predictions.tolist()[0]

new_log = "Your raw log text here"
predicted_details = predict_log_details(new_log)

In [3]:
import pandas as pd

,log,LineId,Date,Time,Level,Component,Content,EventId,EventTemplate
0,"2016-09-28 04:30:30, Info CBS...",1,2016-09-28,04:30:30,Info,CBS,Loaded Servicing Stack v6.1.7601.23505 with Co...,E23,Loaded Servicing Stack <*> with Core: <*>\cbsc...
1,"2016-09-28 04:30:31, Info CSI...",2,2016-09-28,04:30:31,Info,CSI,00000001@2016/9/27:20:30:31.455 WcpInitialize ...,E13,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...
2,"2016-09-28 04:30:31, Info CSI...",3,2016-09-28,04:30:31,Info,CSI,00000002@2016/9/27:20:30:31.458 WcpInitialize ...,E13,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...
3,"2016-09-28 04:30:31, Info CSI...",4,2016-09-28,04:30:31,Info,CSI,00000003@2016/9/27:20:30:31.458 WcpInitialize ...,E13,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...
4,"2016-09-28 04:30:31, Info CBS...",5,2016-09-28,04:30:31,Info,CBS,Ending TrustedInstaller initialization.,E17,Ending TrustedInstaller initialization.
